1. Load the movine data

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error

MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

print(MV_users.head())
print(MV_movies.head())
print(train.head())
print(test.head())

from collections import namedtuple
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)


   uID gender  age  accupation    zip
0    1      F    1          10  48067
1    2      M   56          16  70072
2    3      M   25          15  55117
3    4      M   45           7  02460
4    5      M   25          20  55455
   mID                        title  year  Doc  Com  Hor  Adv  Wes  Dra  Ani  \
0    1                    Toy Story  1995    0    1    0    0    0    0    1   
1    2                      Jumanji  1995    0    0    0    1    0    0    0   
2    3             Grumpier Old Men  1995    0    1    0    0    0    0    0   
3    4            Waiting to Exhale  1995    0    1    0    0    0    1    0   
4    5  Father of the Bride Part II  1995    0    1    0    0    0    0    0   

   ...  Chi  Cri  Thr  Sci  Mys  Rom  Fil  Fan  Act  Mus  
0  ...    1    0    0    0    0    0    0    0    0    0  
1  ...    1    0    0    0    0    0    0    1    0    0  
2  ...    0    0    0    0    0    1    0    0    0    0  
3  ...    0    0    0    0    0    0    0    0    0    

In [17]:


# Read the data
MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

# Create the user-item matrix from the training data
user_item_matrix = train.pivot_table(index='uID', columns='mID', values='rating').fillna(0)

# Apply SVD
n_components = 100  # You can tune this parameter to achieve better performance
svd = TruncatedSVD(n_components)
svd.fit(user_item_matrix)
U = svd.transform(user_item_matrix)
Vt = svd.components_

# Make predictions for the test data
predicted_ratings = np.dot(U, Vt)

# Calculate the RMSE
test_user_indices = [user_item_matrix.index.get_loc(user_id) for user_id in test['uID']]
test_movie_indices = []

for movie_id in test['mID']:
    try:
        movie_index = user_item_matrix.columns.get_loc(movie_id)
    except KeyError:
        movie_index = -1
    test_movie_indices.append(movie_index)

default_rating = train['rating'].mean()
predicted_test_ratings = []

for user_index, movie_index in zip(test_user_indices, test_movie_indices):
    if movie_index == -1:
        predicted_rating = default_rating
    else:
        predicted_rating = predicted_ratings[user_index, movie_index]
    predicted_test_ratings.append(predicted_rating)

# Print the predicted ratings along with their corresponding user and movie IDs
predicted_ratings_df = test.copy()
predicted_ratings_df['predicted_rating'] = predicted_test_ratings
print("\nPredicted Ratings:")
print(predicted_ratings_df)

rmse = np.sqrt(mean_squared_error(test['rating'], predicted_test_ratings))

print(f"RMSE: {rmse}")



Predicted Ratings:
         uID   mID  rating  predicted_rating
0       2233   440       4          2.070383
1       4274   587       5          0.578288
2       2498   454       3          0.725803
3       2868  2336       5          1.632677
4       1636  2686       5          0.521394
...      ...   ...     ...               ...
300058   810   247       4          0.752717
300059  1193  3210       4          0.602386
300060  6039  2289       4         -0.110400
300061  5397   429       3          0.083545
300062  1912   117       4          0.034692

[300063 rows x 4 columns]
RMSE: 3.019331440853742


RMSE, Root Mean Squared Error, measures the difference between the predicted values and the actual values in a regression problem. Usually, a lower RMSE value indicates that the predictions are closer to the actual values. In the case of movie ratings, which typically range from 1 to 5, an RMSE of 3.02 is relatively high. This means that, on average, the predicted ratings differ from the actual ratings by around 3.02 points.

To improve the model's performance, you can try different approaches:

1. Tune the n_components parameter in the TruncatedSVD to find the optimal 
number of latent features that minimize the RMSE.

2. Use other matrix factorization techniques, such as Non-negative Matrix Factorization (NMF) or Probabilistic Matrix Factorization (PMF).